In [8]:
import json
from rapidfuzz import fuzz
from sklearn.metrics import precision_recall_fscore_support


FUZZY_THRESHOLD = 70  

def is_fuzzy_match(pred, gold_set, threshold=FUZZY_THRESHOLD):
    return any(fuzz.ratio(pred, gold) >= threshold for gold in gold_set)

with open(r"C:\Users\bauke\flat_llama_results.json", encoding="utf-8") as f:
    predictions = json.load(f)

with open(r"C:\Users\bauke\OneDrive - KU Leuven\Documents\Documenten\5 digital humanities\stage\Marcel150_clean.json", encoding="utf-8") as f:
    annotated = json.load(f)

gold_entities = {}
for item in annotated:
    filename = item["data"]["filename"]
    entities = item["annotations"][0]["result"]
    gold_per = set()
    gold_loc = set()
    for ent in entities:
        label = ent["value"]["labels"][0]
        text = ent["value"]["text"].strip()
        if label == "PER":
            gold_per.add(text)
        elif label == "LOC":
            gold_loc.add(text)
    gold_entities[filename] = {"PER": gold_per, "LOC": gold_loc}

y_true = []
y_pred = []

for entry in predictions:
    filename = entry["filename"]
    pred_per = set(x.strip() for x in entry.get("people", "").split(",") if x.strip())
    pred_loc = set(x.strip() for x in entry.get("locations", "").split(",") if x.strip())

    gold = gold_entities.get(filename, {"PER": set(), "LOC": set()})
    gold_per = gold["PER"]
    gold_loc = gold["LOC"]

    matched_gold_per = set()
    matched_gold_loc = set()

    for ent in pred_per:
        if is_fuzzy_match(ent, gold_per):
            y_pred.append("PER")
            y_true.append("PER")
            matched_gold_per.add(ent)
        else:
            y_pred.append("PER")
            y_true.append("O")
    for ent in gold_per:
        if not is_fuzzy_match(ent, pred_per):
            y_pred.append("O")
            y_true.append("PER")

    for ent in pred_loc:
        if is_fuzzy_match(ent, gold_loc):
            y_pred.append("LOC")
            y_true.append("LOC")
            matched_gold_loc.add(ent)
        else:
            y_pred.append("LOC")
            y_true.append("O")
    for ent in gold_loc:
        if not is_fuzzy_match(ent, pred_loc):
            y_pred.append("O")
            y_true.append("LOC")

labels = ["PER", "LOC"]
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)

print("\nEntity-Level Evaluation Report (fuzzy matching):")
for i, label in enumerate(labels):
    print(f"{label}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall:    {recall[i]:.4f}")
    print(f"  F1 Score:  {f1[i]:.4f}")
    print(f"  Support:   {support[i]}")



Entity-Level Evaluation Report (fuzzy matching):
PER:
  Precision: 0.3714
  Recall:    0.8375
  F1 Score:  0.5146
  Support:   1188
LOC:
  Precision: 0.2210
  Recall:    0.7024
  F1 Score:  0.3362
  Support:   168
